In [6]:
from pathlib import Path
from sklearn.model_selection import train_test_split
from transformers.trainer_callback import TrainerCallback
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel,AutoModelForSequenceClassification, Trainer, TrainingArguments
import logging
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import pickle

In [7]:
logging.basicConfig(level=logging.DEBUG,format='%(asctime)s %(message)s')

In [8]:
train_data={}
test_data={}
with open('Training Set/train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)
with open('Test Set/test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [18]:
def read_covid_data(data_dic):
    texts = []
    labels = []
    for key in data_dic:
        texts.append(key)
        labels.append(data_dic[key])
    return texts, labels

train_texts, train_labels = read_imdb_split(train_dic)
test_texts, test_labels = read_imdb_split(test_data)

In [19]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [29]:
tokenizer = AutoTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')

In [30]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True,max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True,max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True,max_length=128)

In [31]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [32]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
class PrinterCallback(TrainerCallback):

    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        print(logs)

In [33]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="steps",
    eval_steps=500,
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    load_best_model_at_end=True
)

model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics,
    callbacks=[PrinterCallback]
)

trainer.train()

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

Step,Training Loss,Validation Loss


KeyboardInterrupt: 